### Elasticity and Risk Segmentation
The objective of this notebook is to segment products based on price elasticity and inventory risk, enabling pricing strategies that maximize revenue without creating operational failures.

This notebook:

- Quantifies price elasticity at category level

- Classifies products into elasticity bands

- Combines elasticity with stockout risk

- Identifies safe, risky, and opportunity pricing zones

This ensures pricing decisions are both economically sound and operationally feasible.

### Step 1 — Load Required Data

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("ecommerce_pricing_featured_02.csv")
df.head()

,product_id,category,base_price,competitor_price,promotion_flag,season,customer_segment,day_of_week,channel,current_price,...,fulfilled_minus_10,fulfilled_plus_5,fulfilled_plus_10,feasible_revenue_baseline,feasible_revenue_minus_10,feasible_revenue_plus_5,feasible_revenue_plus_10,stock_minus_10,stock_plus_5,stock_plus_10
0,1102,Electronics,153.93,23.94,True,Fall,Regular,Tue,Web,111.99,...,602.000000,602.000000,602.000000,67417.980000,60676.182000,70788.879000,74159.778000,True,True,True
1,1435,Home,190.17,250.84,False,Spring,Premium,Sat,Mobile App,190.17,...,327.056638,137.952858,74.918265,38221.783673,55976.724816,27546.219808,15671.927082,False,False,False
2,1860,Electronics,166.57,257.71,False,Summer,Regular,Mon,Web,166.57,...,467.862734,302.226637,247.014604,59538.559139,70138.706071,52858.985435,45259.744907,False,False,False
3,1270,Electronics,215.97,238.38,False,Winter,Regular,Mon,Web,215.97,...,173.124550,-41.634578,-113.220954,6468.689904,33650.738241,-9441.410709,-26897.562284,False,False,False
4,1106,Fashion,102.55,255.81,True,Spring,Regular,Mon,Web,81.71,...,571.000000,571.000000,571.000000,46656.410000,41990.769000,48989.230500,51322.051000,True,True,True


### Step 2 — Compute Price Elasticity Proxy

In [ ]:
elasticity_summary= (df.groupby('category').apply(lambda x: x['units_sold'].corr(x['current_price'])).reset_index(name='price_elasticity_proxy'))
elasticity_summary

/tmp/ipython-input-747841637.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  elasticity_summary= (df.groupby('category').apply(lambda x: x['units_sold'].corr(x['current_price'])).reset_index(name='price_elasticity_proxy'))


,category,price_elasticity_proxy
0,Electronics,-0.408791
1,Fashion,-0.401486
2,Grocery,-0.393972
3,Home,-0.412270


#### Interpretation

- Values closer to -1 → highly elastic (price-sensitive)

- Values near 0 → inelastic (price-insensitive)

All categories show negative elasticity, confirming downward-sloping demand

This validates that pricing decisions materially impact demand.

### Step 3 — Define Elasticity Bands
Elasticity Value	Classification
- ≤ -0.40	Highly Elastic
- -0.40 to -0.25	Moderately Elastic
- > -0.25	Inelastic

In [ ]:
def elasticity_band(value):
  if value <= -0.40:
    return "Highly Elastic"
  elif value <= -0.25:
    return "Moderately Elastic"
  else:
    return "Inelastic"
elasticity_summary['elasticity_band']= elasticity_summary['price_elasticity_proxy'].apply(elasticity_band)
elasticity_summary

,category,price_elasticity_proxy,elasticity_band
0,Electronics,-0.408791,Highly Elastic
1,Fashion,-0.401486,Highly Elastic
2,Grocery,-0.393972,Moderately Elastic
3,Home,-0.412270,Highly Elastic


#### Interpretation:
This classification allows pricing teams to:

- Discount only where demand responds

- Raise prices where demand is stable

- Avoid uniform pricing strategies

### Step 4 — Bring in Inventory Risk

We now merge elasticity with stockout risk, because:

Elastic demand + limited inventory = dangerous pricing

In [ ]:
stockout_summary= df.groupby("category").agg({'stock_minus_10': 'mean',
                                              'stock_plus_5': 'mean',
                                              'stock_plus_10': 'mean'}).reset_index()
elasticity_risk_table= elasticity_summary.merge(stockout_summary, on='category', how='left')
elasticity_risk_table

,category,price_elasticity_proxy,elasticity_band,stock_minus_10,stock_plus_5,stock_plus_10
0,Electronics,-0.408791,Highly Elastic,0.339728,0.287060,0.271015
1,Fashion,-0.401486,Highly Elastic,0.328217,0.274351,0.258722
2,Grocery,-0.393972,Moderately Elastic,0.354300,0.296106,0.281985
3,Home,-0.412270,Highly Elastic,0.343981,0.287492,0.273033


### Step 5 — Risk Labeling


In [ ]:
def risk_label(rate):
  if rate < 0.10:
    return "Low"
  elif rate < 0.30:
    return "Medium"
  else:
    return "High"
for col in ['stock_minus_10', "stock_plus_5", "stock_plus_10"]: elasticity_risk_table[col+ '_risk']= elasticity_risk_table[col].apply(risk_label)
elasticity_risk_table

,category,price_elasticity_proxy,elasticity_band,stock_minus_10,stock_plus_5,stock_plus_10,stock_minus_10_risk,stock_plus_5_risk,stock_plus_10_risk
0,Electronics,-0.408791,Highly Elastic,0.339728,0.287060,0.271015,High,Medium,Medium
1,Fashion,-0.401486,Highly Elastic,0.328217,0.274351,0.258722,High,Medium,Medium
2,Grocery,-0.393972,Moderately Elastic,0.354300,0.296106,0.281985,High,Medium,Medium
3,Home,-0.412270,Highly Elastic,0.343981,0.287492,0.273033,High,Medium,Medium


In [ ]:
elasticity_risk_table.to_csv("elasticity_risk_table.csv", index=False)

### What These Results Are Telling Us
Elasticity Results
- Category| Elasticity| Meaning
- Electronics| -0.41| Very price-sensitive
- Fashion| -0.40| Very price-sensitive
- Grocery| -0.39| Moderately price-sensitive
- Home| -0.41| Very price-sensitive

#### Interpretation

- All categories show negative elasticity → demand falls when price rises (expected)

- Electronics, Fashion, and Home are highly elastic

- Grocery is slightly less elastic, but still sensitive

This means:

- Customers strongly respond to price changes across most categories.

So far, this supports discount-led demand growth.

### Inventory Risk Overlay

Now look at stockout risk under pricing scenarios.

#### -10% Price Scenario (Discount)
Category	Stockout Rate	Risk:
- Electronics	34%	High
- Fashion	33%	High
- Grocery	35%	High
- Home	34%	High

Meaning

- Roughly 1 in 3 SKUs would stock out under discounts

- Discount-driven demand exceeds available inventory

- Revenue looks great on paper but is operationally infeasible

#### +5% Price Scenario (Small Increase)
- Category	Stockout Rate	Risk:
All	27–29%	Medium

Meaning

- Slight price increases reduce demand pressure

- Still some inventory stress

- More controllable than aggressive discounting

#### +10% Price Scenario (Aggressive Increase)
- Category	Stockout Rate	Risk:
All	26–28%	Medium

Meaning

- Demand drops significantly

- Inventory pressure eases

- Revenue declines sharply (as seen earlier)

### Strategic Interpretation
#### Key Insight #1 — Elastic ≠ Safe to Discount

Although categories are highly elastic, discounts are dangerous because:

- Elastic demand increases faster than inventory can support

- This creates false revenue optimism

- Stockouts cap realizable revenue

- Elasticity without inventory context is misleading

#### Key Insight #2 — Grocery Is Moderately Elastic, But Still Risky

Grocery:

- Slightly lower elasticity

- Still high stockout risk under discounts

This tells us:

- Even “stable” categories can break under price cuts if supply is tight.

#### Key Insight #3 — Uniform Pricing Is a Mistake

Every category shows:

- Similar elasticity

- Similar stockout behavior

This implies:

- The issue is supply constraint, not demand modeling

- Pricing must be coordinated with inventory, not optimized in isolation

#### Final Strategic Classification
##### Pricing Risk Matrix
Category	Elasticity- Discount Risk	Price-  Increase Safety
- Electronics	High	- ❌ Very risky - Limited
- Fashion	High	- ❌ Very risky	- Limited
- Grocery	Moderate	- ❌ Risky	- Moderate
- Home	High	- ❌ Very risky	- Limited

Although demand is highly price-sensitive across categories, inventory constraints make aggressive discounting operationally infeasible, causing significant stockout risk and unrealizable revenue.